In [ ]:
import torch
import time

def cuda_memory():
    torch.cuda.synchronize()
    torch.cuda.empty_cache()
    time.sleep(0.5)
    print(torch.cuda.memory_allocated(0)) # В байтах

In [ ]:
# Объяснить про неиспользуемые ноды в backward.

In [ ]:
x = torch.zeros(500, 500, dtype=torch.float32, requires_grad=True, device='cuda')
cuda_memory() 
# 1000448 - 500 * 500 * 4 = 448 байт (112 по 4 байта) на requires_grad?

1000448


In [ ]:
a2 = (x + x + x + x).sum()
cuda_memory()
# 1000960 - 1000448 = 512 (128 по 4 байта)

1000960


In [ ]:
a2 = a2.sum()
cuda_memory()
# 1001472 - 1000960 = 512

1000960


In [ ]:
print(x.grad)

None


In [ ]:
a2.backward()
cuda_memory()

2001408


In [ ]:
print(x.grad.shape)

torch.Size([500, 500])


In [ ]:
x.grad = None
cuda_memory()

1000960


In [ ]:
a1 = torch.sin(x + 1) + torch.sin(x + 2) + torch.sin(x + 3) + torch.sin(x + 4)
cuda_memory()

6003200


In [ ]:
a1 = a1.sum()
cuda_memory()

5003264


In [ ]:
a1.backward()
cuda_memory()

2001920


In [ ]:
print(x.grad.shape)

torch.Size([500, 500])


In [ ]:
x.grad = None
cuda_memory()

1001472


In [ ]:
a1 = torch.sin(x + 1) * torch.sin(x + 2) * torch.sin(x + 3) * torch.sin(x + 4)
cuda_memory()

12005888


In [ ]:
a1.mean().backward()
cuda_memory()

3001856


In [ ]:
a1.grad

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  """Entry point for launching an IPython kernel.


In [ ]:
def to_tensors(args):
    return [torch.tensor(x, requires_grad=True, dtype=torch.float32) for x in args]


def print_grad(args):
    for x in args:
        print(round(x.grad.item(), 3), end=' ')
    print()


def print_tensors(args):
    for x in args:
        print(round(x.item(), 3), end=' ')
    print()

In [ ]:
torch.tensor(x, requires_grad=True, dtype=torch.float32)

def v0():
    x, y, z = 1, -1, 2
    x, y, z = to_tensors([x, y, z])
    result = torch.sin(x) * torch.cos(y) * (z + 1) + 2 * x * y * (x + torch.exp(x))
    return [x, y, z], result

[x, y, z], result = v0()
result.backward()
print(round(result.item(), 3))
print_grad([x, y, z])

-6.073
-13.997 9.561 0.455 


In [ ]:
x, y, z = 1, -1, 2
x, y, z = to_tensors([x, y, z])

a1 = torch.sin(x)
a2 = torch.cos(y)
a3 = a1 * a2
a4 = z + 1
a5 = a3 * a4
a6 = 2 * x
a7 = a6 * y
a8 = torch.exp(x)
a9 = x + a8
a10 = a7 * a9
a11 = a5 + a10

print(round(a11.item(), 3))

da11 = 1.0
da10 = da11
da9 = da10 * a7
da8 = da9
da7 = da10 * a9
da6 = da7 * y
da5 = da11
da4 = da5 * a3
da3 = da5 * a4
da2 = da3 * a1
da1 = da3 * a2

dx = da1 * torch.cos(x) + da6 * 2 + da8 * torch.exp(x) + da9
dy = da2 * (-torch.sin(y)) + da7 * a6
dz = da4

print_tensors([dx, dy, dz])

-6.073
-13.997 9.561 0.455 
